In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

sns.set_theme()

# jupyter notebook full-width display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# pandas formatting
pd.set_option('display.float_format', '{:.3f}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', 1000)

In [2]:
def model_kwarg_match(class_dict, kwarg_dict):
    
    df = pd.DataFrame({k: [v] for k, v in class_dict.items()}, index=['MODEL']).T

    df['IMPORT'] = None
    for i, row in df.iterrows():
        try:
            df.loc[i, 'IMPORT'] = kwarg_dict[i]
        except KeyError:
            df.loc[i, 'IMPORT'] = ''
    
    return df


def kwarg_imported(dataframe):
    return dataframe[dataframe.IMPORT != '']


def kwarg_not_imported(dataframe):
    return dataframe[dataframe.IMPORT == '']


def kwarg_foreign(dataframe):
    return dataframe[(dataframe['MODEL'].str.contains('ForeignKey'))]


def kwarg_imp_not_model(dataframe, kwarg_dict):
    for i in kwarg_dict:
        if i in dataframe.index:
            print('FOUND:   ', i)
        else:
            print('---------------\nMISSING:     ', i, '\n---------------')
    return True


def kwarg_not_skippable(dataframe):
    return dataframe[~(dataframe['MODEL'].str.contains('blank=True') | dataframe['MODEL'].str.contains('default') | dataframe['MODEL'].str.contains('property'))]


def kwarg_skippable(dataframe):
    return dataframe[(dataframe['MODEL'].str.contains('blank=True') | dataframe['MODEL'].str.contains('default') | dataframe['MODEL'].str.contains('property'))]


# fish details

In [5]:
# class from dm_apps/herring/models.py
fishdetails_dict = dict(
        
    sample = 'models.ForeignKey(Sample, related_name="fish_details", on_delete=models.CASCADE)',
    fish_number = 'models.IntegerField()',
    fish_length = 'models.FloatField(null=True, blank=True, verbose_name=_("length (mm)"), validators=(MinValueValidator(0),))',
    fish_weight = 'models.FloatField(null=True, blank=True, verbose_name=_("fish weight (g)"), validators=(MinValueValidator(0),))',
    sex = 'models.ForeignKey(Sex, related_name="fish_details", on_delete=models.DO_NOTHING, null=True, blank=True)',
    maturity = 'models.ForeignKey(Maturity, related_name="fish_details", on_delete=models.DO_NOTHING, null=True, blank=True)',
    gonad_weight = 'models.FloatField(null=True, blank=True, verbose_name=_("gonad weight (g)"), validators=(MinValueValidator(0),))',
    parasite = 'models.IntegerField(choices=YESNO_CHOICES, null=True, blank=True)',
    will_count_eggs = 'models.IntegerField(choices=YESNO_CHOICES, null=True, blank=True)',

    # otolith
    annulus_count = 'models.IntegerField(null=True, blank=True)',
    otolith_season = 'models.ForeignKey(OtolithSeason, related_name="fish_details", on_delete=models.DO_NOTHING, null=True, blank=True)',
    otolith_image_remote_filepath = 'models.CharField(max_length=2000, blank=True, null=True)',

    # eggs
    gonad_sub_sample_weight = 'models.FloatField(null=True, blank=True, verbose_name=_("gonad sub-sample weight (g)"))',
    gonad_count = 'models.IntegerField(blank=True, null=True, verbose_name=_("gonad count"))',
    gonad_photo_id = 'models.CharField(max_length=2000, blank=True, null=True, verbose_name=_("photo ID (20 eggs)"))',
    gonad_photo_magnification = 'models.FloatField(blank=True, null=True, verbose_name=_("photo magnification"))',

    test_204_accepted = 'models.CharField(max_length=5, null=True, blank=True)  # ligh_length:fish_weight',
    test_207_accepted = 'models.CharField(max_length=5, null=True, blank=True)  # gonad weight : somatic weight : maturity level',
    test_209_accepted = 'models.CharField(max_length=5, null=True, blank=True)  # number of annuli : fish length',

    # these four fields are deprecated and should be deleted; be sure to delete corresponding tests in the Test table
    test_302_accepted = 'models.CharField(max_length=5, null=True, blank=True)  # fish length within probable range',
    test_305_accepted = 'models.CharField(max_length=5, null=True, blank=True)  # somatic weight within probable range',
    test_308_accepted = 'models.CharField(max_length=5, null=True, blank=True)  # gonad weight within probable range',
    test_311_accepted = 'models.CharField(max_length=5, null=True, blank=True)  # annulus count within probable range',

    remarks = 'models.TextField(null=True, blank=True)',

    # non-editable
    otolith_sampler = 'models.ForeignKey(auth.models.User, on_delete=models.DO_NOTHING, blank=True, null=True, related_name="otolith_sampler_fish_details", editable=False)',
    lab_sampler = 'models.ForeignKey(auth.models.User, on_delete=models.DO_NOTHING, blank=True, null=True, related_name="lab_sampler_fish_details", editable=False)',
    egg_sampler = 'models.ForeignKey(auth.models.User, on_delete=models.DO_NOTHING, blank=True, null=True, related_name="egg_sampler_fish_details", editable=False)',
    creation_date = 'models.DateTimeField(auto_now_add=True, editable=False)',
    last_modified_date = 'models.DateTimeField(auto_now=True, editable=False)',

    created_by = 'models.ForeignKey(auth.models.User, on_delete=models.DO_NOTHING, blank=True, null=True, related_name="created_by_details", editable=False)',
    last_modified_by = 'models.ForeignKey(auth.models.User, on_delete=models.DO_NOTHING, blank=True, null=True, related_name="last_modified_by_details", ditable=False)',

    # calc
    is_empty = 'models.BooleanField(default=True, editable=False)',
    lab_processed_date = 'models.DateTimeField(blank=True, null=True)',
    otolith_processed_date = 'models.DateTimeField(blank=True, null=True)',
    egg_processed_date = 'models.DateTimeField(blank=True, null=True)',
    
    # PROPERTIES
    get_is_empty = '@property ...',
    metadata = '@property ...',
)

fishdetails_kwargs = {x: x for x in list(pd.read_pickle('gaspereau_fishdetails.pickle').columns)}

In [6]:
df_fishdetails = model_kwarg_match(fishdetails_dict, fishdetails_kwargs)

print('\n=======================================\nIMPORTED')
display(kwarg_imported(df_fishdetails))
print('\n\n=======================================\nNOT IMPORTED')
display(kwarg_not_imported(df_fishdetails))
print('\n\n=======================================\nFOREIGN KEYS')
display(kwarg_foreign(df_fishdetails))
print('\n\n=======================================\nNOT SKIPPABLE')
display(kwarg_not_skippable(df_fishdetails))
print('\n\n=======================================\nSKIPPABLE')
display(kwarg_skippable(df_fishdetails))


IMPORTED


,MODEL,IMPORT
fish_number,models.IntegerField(),fish_number
fish_length,"models.FloatField(null=True, blank=True, verbose_name=_(""length (mm)""), validators=(MinValueValidator(0),))",fish_length
fish_weight,"models.FloatField(null=True, blank=True, verbose_name=_(""fish weight (g)""), validators=(MinValueValidator(0),))",fish_weight
gonad_weight,"models.FloatField(null=True, blank=True, verbose_name=_(""gonad weight (g)""), validators=(MinValueValidator(0),))",gonad_weight




NOT IMPORTED


,MODEL,IMPORT
sample,"models.ForeignKey(Sample, related_name=""fish_details"", on_delete=models.CASCADE)",
sex,"models.ForeignKey(Sex, related_name=""fish_details"", on_delete=models.DO_NOTHING, null=True, blank=True)",
maturity,"models.ForeignKey(Maturity, related_name=""fish_details"", on_delete=models.DO_NOTHING, null=True, blank=True)",
parasite,"models.IntegerField(choices=YESNO_CHOICES, null=True, blank=True)",
will_count_eggs,"models.IntegerField(choices=YESNO_CHOICES, null=True, blank=True)",
annulus_count,"models.IntegerField(null=True, blank=True)",
otolith_season,"models.ForeignKey(OtolithSeason, related_name=""fish_details"", on_delete=models.DO_NOTHING, null=True, blank=True)",
otolith_image_remote_filepath,"models.CharField(max_length=2000, blank=True, null=True)",
gonad_sub_sample_weight,"models.FloatField(null=True, blank=True, verbose_name=_(""gonad sub-sample weight (g)""))",
gonad_count,"models.IntegerField(blank=True, null=True, verbose_name=_(""gonad count""))",




FOREIGN KEYS


,MODEL,IMPORT
sample,"models.ForeignKey(Sample, related_name=""fish_details"", on_delete=models.CASCADE)",
sex,"models.ForeignKey(Sex, related_name=""fish_details"", on_delete=models.DO_NOTHING, null=True, blank=True)",
maturity,"models.ForeignKey(Maturity, related_name=""fish_details"", on_delete=models.DO_NOTHING, null=True, blank=True)",
otolith_season,"models.ForeignKey(OtolithSeason, related_name=""fish_details"", on_delete=models.DO_NOTHING, null=True, blank=True)",
otolith_sampler,"models.ForeignKey(auth.models.User, on_delete=models.DO_NOTHING, blank=True, null=True, related_name=""otolith_sampler_fish_details"", editable=False)",
lab_sampler,"models.ForeignKey(auth.models.User, on_delete=models.DO_NOTHING, blank=True, null=True, related_name=""lab_sampler_fish_details"", editable=False)",
egg_sampler,"models.ForeignKey(auth.models.User, on_delete=models.DO_NOTHING, blank=True, null=True, related_name=""egg_sampler_fish_details"", editable=False)",
created_by,"models.ForeignKey(auth.models.User, on_delete=models.DO_NOTHING, blank=True, null=True, related_name=""created_by_details"", editable=False)",
last_modified_by,"models.ForeignKey(auth.models.User, on_delete=models.DO_NOTHING, blank=True, null=True, related_name=""last_modified_by_details"", ditable=False)",




NOT SKIPPABLE


,MODEL,IMPORT
sample,"models.ForeignKey(Sample, related_name=""fish_details"", on_delete=models.CASCADE)",
fish_number,models.IntegerField(),fish_number
creation_date,"models.DateTimeField(auto_now_add=True, editable=False)",
last_modified_date,"models.DateTimeField(auto_now=True, editable=False)",




SKIPPABLE


,MODEL,IMPORT
fish_length,"models.FloatField(null=True, blank=True, verbose_name=_(""length (mm)""), validators=(MinValueValidator(0),))",fish_length
fish_weight,"models.FloatField(null=True, blank=True, verbose_name=_(""fish weight (g)""), validators=(MinValueValidator(0),))",fish_weight
sex,"models.ForeignKey(Sex, related_name=""fish_details"", on_delete=models.DO_NOTHING, null=True, blank=True)",
maturity,"models.ForeignKey(Maturity, related_name=""fish_details"", on_delete=models.DO_NOTHING, null=True, blank=True)",
gonad_weight,"models.FloatField(null=True, blank=True, verbose_name=_(""gonad weight (g)""), validators=(MinValueValidator(0),))",gonad_weight
parasite,"models.IntegerField(choices=YESNO_CHOICES, null=True, blank=True)",
will_count_eggs,"models.IntegerField(choices=YESNO_CHOICES, null=True, blank=True)",
annulus_count,"models.IntegerField(null=True, blank=True)",
otolith_season,"models.ForeignKey(OtolithSeason, related_name=""fish_details"", on_delete=models.DO_NOTHING, null=True, blank=True)",
otolith_image_remote_filepath,"models.CharField(max_length=2000, blank=True, null=True)",


In [8]:
kwarg_imp_not_model(df_fishdetails, fishdetails_kwargs)

---------------
MISSING:      sample_id 
---------------
---------------
MISSING:      DATETIME 
---------------
---------------
MISSING:      SITE 
---------------
---------------
MISSING:      CONDITION 
---------------
FOUND:    fish_number
---------------
MISSING:      FL_WET 
---------------
---------------
MISSING:      FL_FROZEN 
---------------
FOUND:    fish_length
FOUND:    fish_weight
---------------
MISSING:      SPECIES 
---------------
---------------
MISSING:      SEX 
---------------
---------------
MISSING:      maturity_id 
---------------
FOUND:    gonad_weight
---------------
MISSING:      AGING_PERSON_1 
---------------
---------------
MISSING:      AGE_1 
---------------
---------------
MISSING:      FSP_1 
---------------
---------------
MISSING:      AGE_COMMENTS_1 
---------------
---------------
MISSING:      AGING_PERSON_2 
---------------
---------------
MISSING:      AGE_2 
---------------
---------------
MISSING:      FSP_2 
---------------
---------------

True

# samples

In [9]:
# class from dm_apps/herring/models.py
sample_dict = dict(
    SAMPLE_TYPE_CHOICES = '((1, "Port"),(2, "Sea"),)',
    season_type_choices = '((1, "spring"),(2, "fall"),)',
    type = 'models.IntegerField(blank=True, null=True, choices=SAMPLE_TYPE_CHOICES, verbose_name=_("Sample type"))',

    species = 'models.ForeignKey(Species, related_name="samples", on_delete=models.DO_NOTHING, blank=True, null=True)',
    sample_date = 'models.DateTimeField()',
    sampler_ref_number = 'models.IntegerField(verbose_name="field reference number", blank=True, null=True, help_text="This might be a sampler\'s reference number or a set number from a survey")',
    sampler = 'models.ForeignKey(Sampler, related_name="samples", on_delete=models.DO_NOTHING, null=True, blank=True)',

    port = 'models.ForeignKey(shared_models.Port, related_name="herring_samples", on_delete=models.DO_NOTHING, null=True, blank=True)',
    survey_id = 'models.CharField(max_length=50, null=True, blank=True, verbose_name="survey identifier",help_text="Usually the mission number of the survey from which the sample came.")',
    latitude_n = 'models.FloatField(null=True, blank=True, verbose_name="Latitude", help_text="In decimal degrees, e.g., 48.67789")',
    longitude_w = 'models.FloatField(null=True, blank=True, verbose_name="Longitude", help_text="In decimal degrees, e.g., -64.57819")',
    fishing_area = 'models.ForeignKey(FishingArea, related_name="samples", on_delete=models.DO_NOTHING, null=True, blank=True)',
    gear = 'models.ForeignKey(Gear, related_name="samples", on_delete=models.DO_NOTHING, null=True, blank=True)',
    experimental_net_used = 'models.IntegerField(choices=YESNO_CHOICES, null=True, blank=True)',
    vessel_cfvn = 'models.IntegerField(null=True, blank=True, verbose_name=_("vessel CFVN"))',
    mesh_size = 'models.ForeignKey(MeshSize, related_name="samples", on_delete=models.DO_NOTHING, null=True, blank=True)',
    catch_weight_lbs = 'models.FloatField(null=True, blank=True, verbose_name="Catch weight (lbs)", help_text="Total weight of the catch from which the sample was collected")',
    sample_weight_lbs = 'models.FloatField(null=True, blank=True, verbose_name="Sample weight (lbs)", help_text="Total weight of the sample collected")',
    total_fish_measured = 'models.IntegerField(null=True, blank=True, help_text="How many specimens were measured?")',
    total_fish_preserved = 'models.IntegerField(null=True, blank=True, help_text="How many specimens were collected?")',
    remarks = 'models.TextField(null=True, blank=True)',

    # not editable
    district = 'models.ForeignKey(District, related_name="samples", on_delete=models.DO_NOTHING, null=True, blank=True, editable=False)  # this field should be deleted',
    old_id = 'models.CharField(max_length=100, null=True, blank=True, editable=False)',
    season = 'models.IntegerField(null=True, blank=True, verbose_name=_("year"), editable=False)',
    season_type = 'models.IntegerField(null=True, blank=True, choices=season_type_choices, editable=False, verbose_name=_("season"))',
    length_frequencies = 'models.ManyToManyField(to=LengthBin, through="LengthFrequency", editable=False)',
    lab_processing_complete = 'models.BooleanField(default=False, editable=False)',
    otolith_processing_complete = 'models.BooleanField(default=False, editable=False)',
    egg_processing_complete = 'models.BooleanField(default=False, editable=False)',

    created_by = 'models.ForeignKey(User, on_delete=models.DO_NOTHING, blank=True, null=True, related_name="created_by_samples", editable=False)',
    last_modified_by = 'models.ForeignKey(User, on_delete=models.DO_NOTHING, blank=True, null=True, related_name="last_modified_by_samples", editable=False)',
    creation_date = 'models.DateTimeField(auto_now_add=True, editable=False)',
    last_modified_date = 'models.DateTimeField(auto_now=True, editable=False)',
    
    # PROPERTIES
    lab_complete_fish = '@property ...',
    oto_complete_fish = '@property ...',
    egg_complete_fish = '@property ...',
    total_egg_fish = '@property ...',
    is_complete = '@property ...',
    lf_count = '@property ...'
)

gaspereau_sample_columns = [
    "r['id']",
    "r['sample_date']",
    "r['district_id']",
    "r['fishing_area_id']",
    "r['SAMPLER']",
    "r['CODE']",
    "r['gear_id']",
    "r['SITE']",
    "r['NO_NETS']",
    "r['catch_weight_lbs']",
    "r['HOURS_FISHED']",
    "r['ZONE']",
    "r['LAST_NAME']",
    "r['COMMENTS']",
    "r['BYCATCH_SBASS']",
    "r['BYCATCH_SHAD']",
    "r['BYCATCH_OTHER']",
    "r['sample_weight_lbs']"
]


# import script
sample_kwargs = dict(
    sample_id = "r['id']",
    district_id = "r['district_id']",
    fishing_area_id = "r['fishing_area_id']",
    SAMPLER = "r['SAMPLER']",
    CODE = "r['CODE']",
    gear_id = "r['gear_id']",
    SITE = "r['SITE']",
    NO_NETS = "r['NO_NETS']",
    HOURS_FISHED = "r['HOURS_FISHED']",
    ZONE = "r['ZONE']",
    LAST_NAME = "r['LAST_NAME']",
    COMMENTS = "r['COMMENTS']",
    BYCATCH_SBASS = "r['BYCATCH_SBASS']",
    BYCATCH_SHAD = "r['BYCATCH_SHAD']",
    BYCATCH_OTHER = "r['BYCATCH_OTHER']",
    
    
    
    SAMPLE_TYPE_CHOICES = '',
    season_type_choices = '',
    type = '',
 
    species = '',
    sample_date = "r['sample_date']",
    sampler_ref_number = '',
    sampler = '',
 
    port = '',
    survey_id = '',
    latitude_n = '',
    longitude_w = '',
    fishing_area = '',
    gear = '',
    experimental_net_used = '',
    vessel_cfvn = '',
    mesh_size = '',
    catch_weight_lbs = "r['catch_weight_lbs']",
    sample_weight_lbs = "r['sample_weight_lbs']",
    total_fish_measured = '',
    total_fish_preserved = '',
    remarks = '',
 
    # not editable
    district = '',
    old_id = '',
    season = '',
    season_type = '',
    length_frequencies = '',
    lab_processing_complete = '',
    otolith_processing_complete = '',
    egg_processing_complete = '',
 
    created_by = '',
    last_modified_by = '',
    creation_date = '',
    last_modified_date = '',
    
    # PROPERTIES
    lab_complete_fish = '',
    oto_complete_fish = '',
    egg_complete_fish = '',
    total_egg_fish = '',
    is_complete = '',
    lf_count = ''

)

In [10]:
df_samp = model_kwarg_match(sample_dict, sample_kwargs)

print('\n=======================================\nIMPORTED')
display(kwarg_imported(df_samp))
print('\n\n=======================================\nNOT IMPORTED')
display(kwarg_not_imported(df_samp))
print('\n\n=======================================\nFOREIGN KEYS')
display(kwarg_foreign(df_samp))
print('\n\n=======================================\nNOT SKIPPABLE')
display(kwarg_not_skippable(df_samp))
print('\n\n=======================================\nSKIPPABLE')
display(kwarg_skippable(df_samp))


IMPORTED


,MODEL,IMPORT
sample_date,models.DateTimeField(),r['sample_date']
catch_weight_lbs,"models.FloatField(null=True, blank=True, verbose_name=""Catch weight (lbs)"", help_text=""Total weight of the catch from which the sample was collected"")",r['catch_weight_lbs']
sample_weight_lbs,"models.FloatField(null=True, blank=True, verbose_name=""Sample weight (lbs)"", help_text=""Total weight of the sample collected"")",r['sample_weight_lbs']




NOT IMPORTED


,MODEL,IMPORT
SAMPLE_TYPE_CHOICES,"((1, ""Port""),(2, ""Sea""),)",
season_type_choices,"((1, ""spring""),(2, ""fall""),)",
type,"models.IntegerField(blank=True, null=True, choices=SAMPLE_TYPE_CHOICES, verbose_name=_(""Sample type""))",
species,"models.ForeignKey(Species, related_name=""samples"", on_delete=models.DO_NOTHING, blank=True, null=True)",
sampler_ref_number,"models.IntegerField(verbose_name=""field reference number"", blank=True, null=True, help_text=""This might be a sampler's reference number or a set number from a survey"")",
sampler,"models.ForeignKey(Sampler, related_name=""samples"", on_delete=models.DO_NOTHING, null=True, blank=True)",
port,"models.ForeignKey(shared_models.Port, related_name=""herring_samples"", on_delete=models.DO_NOTHING, null=True, blank=True)",
survey_id,"models.CharField(max_length=50, null=True, blank=True, verbose_name=""survey identifier"",help_text=""Usually the mission number of the survey from which the sample came."")",
latitude_n,"models.FloatField(null=True, blank=True, verbose_name=""Latitude"", help_text=""In decimal degrees, e.g., 48.67789"")",
longitude_w,"models.FloatField(null=True, blank=True, verbose_name=""Longitude"", help_text=""In decimal degrees, e.g., -64.57819"")",




FOREIGN KEYS


,MODEL,IMPORT
species,"models.ForeignKey(Species, related_name=""samples"", on_delete=models.DO_NOTHING, blank=True, null=True)",
sampler,"models.ForeignKey(Sampler, related_name=""samples"", on_delete=models.DO_NOTHING, null=True, blank=True)",
port,"models.ForeignKey(shared_models.Port, related_name=""herring_samples"", on_delete=models.DO_NOTHING, null=True, blank=True)",
fishing_area,"models.ForeignKey(FishingArea, related_name=""samples"", on_delete=models.DO_NOTHING, null=True, blank=True)",
gear,"models.ForeignKey(Gear, related_name=""samples"", on_delete=models.DO_NOTHING, null=True, blank=True)",
mesh_size,"models.ForeignKey(MeshSize, related_name=""samples"", on_delete=models.DO_NOTHING, null=True, blank=True)",
district,"models.ForeignKey(District, related_name=""samples"", on_delete=models.DO_NOTHING, null=True, blank=True, editable=False) # this field should be deleted",
created_by,"models.ForeignKey(User, on_delete=models.DO_NOTHING, blank=True, null=True, related_name=""created_by_samples"", editable=False)",
last_modified_by,"models.ForeignKey(User, on_delete=models.DO_NOTHING, blank=True, null=True, related_name=""last_modified_by_samples"", editable=False)",




NOT SKIPPABLE


,MODEL,IMPORT
SAMPLE_TYPE_CHOICES,"((1, ""Port""),(2, ""Sea""),)",
season_type_choices,"((1, ""spring""),(2, ""fall""),)",
sample_date,models.DateTimeField(),r['sample_date']
length_frequencies,"models.ManyToManyField(to=LengthBin, through=""LengthFrequency"", editable=False)",
creation_date,"models.DateTimeField(auto_now_add=True, editable=False)",
last_modified_date,"models.DateTimeField(auto_now=True, editable=False)",




SKIPPABLE


,MODEL,IMPORT
type,"models.IntegerField(blank=True, null=True, choices=SAMPLE_TYPE_CHOICES, verbose_name=_(""Sample type""))",
species,"models.ForeignKey(Species, related_name=""samples"", on_delete=models.DO_NOTHING, blank=True, null=True)",
sampler_ref_number,"models.IntegerField(verbose_name=""field reference number"", blank=True, null=True, help_text=""This might be a sampler's reference number or a set number from a survey"")",
sampler,"models.ForeignKey(Sampler, related_name=""samples"", on_delete=models.DO_NOTHING, null=True, blank=True)",
port,"models.ForeignKey(shared_models.Port, related_name=""herring_samples"", on_delete=models.DO_NOTHING, null=True, blank=True)",
survey_id,"models.CharField(max_length=50, null=True, blank=True, verbose_name=""survey identifier"",help_text=""Usually the mission number of the survey from which the sample came."")",
latitude_n,"models.FloatField(null=True, blank=True, verbose_name=""Latitude"", help_text=""In decimal degrees, e.g., 48.67789"")",
longitude_w,"models.FloatField(null=True, blank=True, verbose_name=""Longitude"", help_text=""In decimal degrees, e.g., -64.57819"")",
fishing_area,"models.ForeignKey(FishingArea, related_name=""samples"", on_delete=models.DO_NOTHING, null=True, blank=True)",
gear,"models.ForeignKey(Gear, related_name=""samples"", on_delete=models.DO_NOTHING, null=True, blank=True)",


In [11]:
kwarg_imp_not_model(df_samp, sample_kwargs)

---------------
MISSING:      sample_id 
---------------
---------------
MISSING:      district_id 
---------------
---------------
MISSING:      fishing_area_id 
---------------
---------------
MISSING:      SAMPLER 
---------------
---------------
MISSING:      CODE 
---------------
---------------
MISSING:      gear_id 
---------------
---------------
MISSING:      SITE 
---------------
---------------
MISSING:      NO_NETS 
---------------
---------------
MISSING:      HOURS_FISHED 
---------------
---------------
MISSING:      ZONE 
---------------
---------------
MISSING:      LAST_NAME 
---------------
---------------
MISSING:      COMMENTS 
---------------
---------------
MISSING:      BYCATCH_SBASS 
---------------
---------------
MISSING:      BYCATCH_SHAD 
---------------
---------------
MISSING:      BYCATCH_OTHER 
---------------
FOUND:    SAMPLE_TYPE_CHOICES
FOUND:    season_type_choices
FOUND:    type
FOUND:    species
FOUND:    sample_date
FOUND:    sampler_ref_number
F

True

# herring table fields

In [12]:
# fish details 
sorted(list(fishdetails_dict.keys()))

['annulus_count',
 'created_by',
 'creation_date',
 'egg_processed_date',
 'egg_sampler',
 'fish_length',
 'fish_number',
 'fish_weight',
 'get_is_empty',
 'gonad_count',
 'gonad_photo_id',
 'gonad_photo_magnification',
 'gonad_sub_sample_weight',
 'gonad_weight',
 'is_empty',
 'lab_processed_date',
 'lab_sampler',
 'last_modified_by',
 'last_modified_date',
 'maturity',
 'metadata',
 'otolith_image_remote_filepath',
 'otolith_processed_date',
 'otolith_sampler',
 'otolith_season',
 'parasite',
 'remarks',
 'sample',
 'sex',
 'test_204_accepted',
 'test_207_accepted',
 'test_209_accepted',
 'test_302_accepted',
 'test_305_accepted',
 'test_308_accepted',
 'test_311_accepted',
 'will_count_eggs']

In [13]:
kwarg_not_skippable(model_kwarg_match(fishdetails_dict, {}))

,MODEL,IMPORT
sample,"models.ForeignKey(Sample, related_name=""fish_details"", on_delete=models.CASCADE)",
fish_number,models.IntegerField(),
creation_date,"models.DateTimeField(auto_now_add=True, editable=False)",
last_modified_date,"models.DateTimeField(auto_now=True, editable=False)",


In [16]:
# sample
sorted(list(sample_dict.keys()))

['SAMPLE_TYPE_CHOICES',
 'catch_weight_lbs',
 'created_by',
 'creation_date',
 'district',
 'egg_complete_fish',
 'egg_processing_complete',
 'experimental_net_used',
 'fishing_area',
 'gear',
 'is_complete',
 'lab_complete_fish',
 'lab_processing_complete',
 'last_modified_by',
 'last_modified_date',
 'latitude_n',
 'length_frequencies',
 'lf_count',
 'longitude_w',
 'mesh_size',
 'old_id',
 'oto_complete_fish',
 'otolith_processing_complete',
 'port',
 'remarks',
 'sample_date',
 'sample_weight_lbs',
 'sampler',
 'sampler_ref_number',
 'season',
 'season_type',
 'season_type_choices',
 'species',
 'survey_id',
 'total_egg_fish',
 'total_fish_measured',
 'total_fish_preserved',
 'type',
 'vessel_cfvn']

In [15]:
kwarg_not_skippable(model_kwarg_match(sample_dict, {}))

,MODEL,IMPORT
SAMPLE_TYPE_CHOICES,"((1, ""Port""),(2, ""Sea""),)",
season_type_choices,"((1, ""spring""),(2, ""fall""),)",
sample_date,models.DateTimeField(),
length_frequencies,"models.ManyToManyField(to=LengthBin, through=""LengthFrequency"", editable=False)",
creation_date,"models.DateTimeField(auto_now_add=True, editable=False)",
last_modified_date,"models.DateTimeField(auto_now=True, editable=False)",
